# Import Modules & Packages

In [1]:
import random
import smtplib
import yaml

# Define Functions

In [36]:
def send_email(sender: str,
               sndr_pswd: str,
               to: str,
               subject: str,
               msg: str) -> None:
    '''
    Sends automated email message. (Assumes G-mail.)
    
    Args:
        sender: Sender's email address.
        sndr_pswd: Sender's email account password.
        to: Recipient email address.
        subject: Subject line of email.
        msg: Message body of email.
    
    Returns:
        None
    '''
    try:
        server = smtplib.SMTP('smtp.gmail.com:587')
        server.ehlo()
        server.starttls()
        server.login(sender, sndr_pswd)
        message = 'Subject: {}\n\n{}'.format(subject, msg)
        server.sendmail(sender, to, message)
        server.quit()
        print("Success: Email sent!")
    except:
        print("Email failed to send.")
    return None

In [37]:
def read_config(config_file: str) -> dict:
    '''
    Reads the elements of a YAML configuration file.
    
    Args:
        config_file: Filename (and path) to configuration file.
    
    Returns:
        cfg: Dictionary of configuration file enteries
    '''
    
    with open(config_file,"r") as file:
        cfg = yaml.safe_load(file)
        
    return cfg

In [38]:
def dict_key_to_list(dictionary):
    '''
    DEPRECATED:
    Creates list from hierarchal/multi-tiered dictionary keys. 
    Primarily intended for dictionaries generated by the 'read_config'
    function for people names.
    
    Args:
    dictionary: dictionary
    
    Returns:
    list of key values
    '''
    
    # Empty list
    dict_list = []
    
    for key,value in dictionary.items():
        temp = key
        dict_list.append(temp)

    return dict_list 

In [39]:
def person_match(config_people: str) -> dict:
    '''
    Randomly matches a person with another person.
    This function allows for a dictionary entry to exclude 
    another person should that be required.
    
    This function will run until all participants are assigned
    a person.
    
    Args:
        config_people: Input configuration file that contains names and email addresses of participants.
    
    Returns:
        new_dict: Output dictionary that contains which person is matched to another person.
    '''
    # Read people config file
    ppl_dict = read_config(config_file=config_people)
    
    # Set run condition
    run_condition = True
    while run_condition:
        new_dict = {}
        assigned = []
        for p in ppl_dict.keys():
            recips = []
            if 'exclude' in ppl_dict[p]:
                exclude = ppl_dict[p]['exclude']
                exclude = list(exclude.split(","))
                recips = list(set(ppl_dict.keys()) - set(exclude) - set(assigned))
            else:
                recips = list(set(ppl_dict.keys()) - set(assigned))

            try:
                recips.remove(p)
            except ValueError:
                pass

            try:
                rand = random.randint(0, (len(recips))-1)
                random_recip = recips[rand]
                assigned.append(random_recip)
                tmp_dict = {p: random_recip}
                new_dict.update(tmp_dict)
                del tmp_dict
                run_condition = False
            except ValueError:
                run_condition = True
                pass
    return new_dict

In [41]:
def send_test_email(config_file: str,
                    config_people: str,
                    dry_run: bool = False
                   ) -> None:
    '''
    Sends test email to the email addesses in the 'config_people' config file.
    
    Args:
        config_file: Configuration file that contains login credentials for (throwaway) G-mail account
        config_people: Configuration file that contains names, and email addresses (and exclusion list)
        dry_run: If true, names and recipients are printed. If false, then emails are sent.
    
    Returns:
        None
    '''
    
    auto = read_config(config_file=config_file)
    ppl_dict = read_config(config_file=config_people)
    
    for p in ppl_dict.keys():
        if dry_run is True:
            print(f"Sent test email to {p}")
        else:
            msg = f"Hi {p}, \
            \n\n \
            This is a test email. \
            \n\n \
            Please do not reply to this email."
            subject = f"Test email - Family Secret Santa Gift Exchange"
            send_email(auto['email'],auto['password'],
                       ppl_dict[p]['email'],subject,msg)
    return None

In [28]:
def gift_exchange(config_file, config_people, dry_run = False, verbose = False, budget = 15.00, year = 2019):
    '''
    DEPRECATED:
    Randomized gift exchange function. Randomizes list of people and
    matches one person to another.
    
    Args:
    config_file: configuration file that contains login credentials for (throwaway) G-mail account
    config_people: configuration file that contains names, and email addresses (and exclusion list)
    dry_run: boolean - if true, names and recipients are printed. If false, then emails are sent.
    verbose: boolean - if true, prints additional information to screen.
    budget: float - dollar amount for the budget
    year: int - year of the gift exchange
    
    Returns:
    None
    '''
    
    auto = read_config(cfg_file)
    ppl_dict = read_config(ppl_file)
    
    budget = str(budget)
    year = str(year)
    
    # Participants
    people = dict_key_to_list(ppl_dict)

    # Empty list to store the people who've already had an assigment made
    assigned = []
    assigned_dict = {}
    
    # For each person, assemble a list of possible recipients.
    for p in people:

        # Init a list to store potential recipients in
        recips = []

        # Remove already assigned people from potential recipients
        # recips = list(set(people) - set(assigned))

        if 'exclude' in ppl_dict[p]:
            exclude = ppl_dict[p]['exclude']
            exclude = list(exclude.split(","))
            recips = list(set(people) - set(exclude) - set(assigned))
        else:
            recips = list(set(people) - set(assigned))

        # Don't let a user give to themselves
        try:
            recips.remove(p)
        except ValueError:
            pass

        # Now grab a random person from the remaining recipients
        rand = random.randint(0, (len(recips))-1)
        random_recip = recips[rand]

        # Assign that recipient to the "assigned" list so they don't appear again
        assigned.append(random_recip)

        if dry_run is True:
            print(f"{p} gives to {random_recip}")
        else:
            msg = f"Hi {p}, \n\n You are giving to {random_recip}. \
            \n\n The recommended budget is ${budget}, but please feel free to spend whatever is appropriate. \
            \n\n Take care. \n\n -- \n\n This is an automated email. \n\n Please, do not reply to this email."
            subject = f"Family Secret Santa Gift Exchange {year}"
            send_email(auto['email'],auto['password'],
                       ppl_dict[p]['email'],subject,msg)
            if verbose is True:
                print(f"Sent email to {p}")

In [42]:
def gift_exchange(config_file: str,
                  config_people: str,
                  dry_run: bool = False,
                  test_email: bool = False,
                  verbose: bool = False,
                  budget: float = 15.00,
                  year: int = 2020
                 ) -> None:
    '''
    Randomized gift exchange function. Randomizes list of people and
    matches one person to another.
    
    Args:
        config_file: Configuration file that contains login credentials for (throwaway) G-mail account
        config_people: Configuration file that contains names, and email addresses (and exclusion list)
        dry_run: If true, names and recipients are printed. If false, then emails are sent.
        test_email: Sends test email to each person.
        verbose: If true, prints additional information to screen.
        budget: Dollar amount for the budget
        year: Year of the gift exchange
    
    Returns:
        None
    '''
    
    auto = read_config(config_file=config_file)
    ppl_dict = read_config(config_file=config_people)
    
    match_dict = person_match(config_people=config_people)
    
    # For each person, assemble a list of possible recipients.
    for p in ppl_dict.keys():
        random_recip = match_dict[p]
        if test_email:
            send_test_email(config_file=config_file,
                            config_people=config_people,
                            dry_run=dry_run)
        elif dry_run:
            print(f"{p} gives to {random_recip}")
        else:
            msg = f"Hi {p}, \n\n \
            You are giving to {random_recip}. \
            \n\n \
            The recommended budget is ${budget}, \
            but please feel free to spend whatever is appropriate. \
            \n\n \
            Take care. \
            \n\n \
            -- \
            \n\n \
            This is an automated email. \
            \n\n \
            Please, do not reply to this email."
            subject = f"Family Secret Santa Gift Exchange {year}"
            send_email(sender=auto['email'],
                       sndr_pswd=auto['password'],
                       to=ppl_dict[p]['email'],
                       subject=subject,
                       msg=msg)
            if verbose is True:
                print(f"Sent email to {p}")
    return None

In [7]:
cfg_file = "config.gift.yml"
ppl_file = "config.people.yml"

# cfg_file = "orig_files/config.gift.yml"
# ppl_file = "orig_files/config.people.yml"

# Gift Exchange
-----

## Send Test e-mails

In [8]:
send_test_email(cfg_file,ppl_file,dry_run=True)
# send_test_email(cfg_file,ppl_file,dry_run=False)

Sent test email to Person1
Sent test email to Person2
Sent test email to Person3
Sent test email to Person4


## Send emails for Secret Santa

In [31]:
gift_exchange(cfg_file,ppl_file,dry_run=True,budget=15,year=2019)
# gift_exchange(cfg_file,ppl_file,dry_run=False,verbose=False,budget=15,year=2019)

Person1 gives to Person3
Person2 gives to Person1
Person3 gives to Person2


ValueError: empty range for randrange() (0,0, 0)

In [27]:
run_condition = True
while run_condition:
    try:
        gift_exchange(cfg_file,ppl_file,dry_run=True,budget=15,year=2019)
        run_condition = False
    except ValueError:
        run_condition = True
        continue
    break

Person1 gives to Person3
Person2 gives to Person1
Person3 gives to Person2
Person1 gives to Person4
Person2 gives to Person1
Person3 gives to Person2
Person4 gives to Person3
